### 1. Monte um passo a passo para o Bagging:

1. **Coleta de Dados**: 
    - Tenha um conjunto de dados de treinamento.
2. **Criação de Amostras de Bootstrap**:
    - Gere múltiplas amostras de bootstrap a partir do conjunto de dados original. Cada amostra é criada selecionando aleatoriamente instâncias do conjunto de dados com reposição. Isso significa que alguns exemplos podem aparecer várias vezes em uma amostra, enquanto outros podem não aparecer.

3. **Treinamento dos Modelos Base**:
    - Treine um modelo base (como uma árvore de decisão) em cada amostra de bootstrap gerada. O tipo de modelo base pode variar, mas frequentemente são usados modelos simples e instáveis como árvores de decisão.

4. **Agregação dos Modelos**:
    - Após o treinamento, cada modelo base faz previsões independentes em novos dados.

5. **Combinação das Previsões**:
    - Para problemas de regressão, a previsão final é a média das previsões de todos os modelos base.
    - Para problemas de classificação, a previsão final é determinada pelo voto da maioria das previsões dos modelos base.

6. **Avaliação do Modelo**:
    - Avalie o desempenho do modelo combinado (ensemble) utilizando métricas apropriadas (como acurácia, precisão, recall, F1-score para classificação, ou MSE para regressão).

### 2. Explique com suas palavras o Bagging:

Bagging, ou Bootstrap Aggregating, é uma técnica de machine learning que melhora a precisão e a robustez de modelos preditivos. A ideia principal por trás do Bagging é reduzir a variância de um modelo base instável (como uma árvore de decisão) através da criação de múltiplas versões do modelo, cada uma treinada em diferentes subconjuntos de dados de treinamento obtidos por meio de amostragem com reposição (bootstrap).

Cada um desses subconjuntos (ou amostras de bootstrap) é usado para treinar um modelo base independente. Esses modelos base são então combinados de forma a agregar suas previsões: para problemas de regressão, as previsões são combinadas tirando a média; para problemas de classificação, é utilizado o voto da maioria. 

O Bagging funciona bem para modelos que têm alta variância, ou seja, que mudam significativamente com pequenas mudanças nos dados de treinamento. Ao combinar vários modelos, o Bagging tende a suavizar as flutuações e, assim, melhorar o desempenho geral e a estabilidade da previsão.

Por exemplo, o Random Forest, que é uma extensão do Bagging, várias árvores de decisão são treinadas em amostras de bootstrap e uma pequena variação adicional é introduzida ao selecionar aleatoriamente subconjuntos de características em cada divisão da árvore, o que ajuda a decorrelacionar as árvores e aumenta a precisão do ensemble.

In [5]:
import pandas as pd
import numpy as np

from sklearn.datasets        import load_iris
from sklearn.datasets        import load_diabetes

from sklearn.model_selection import train_test_split

from sklearn.tree            import DecisionTreeClassifier
from sklearn.metrics         import accuracy_score

from sklearn.tree            import DecisionTreeRegressor
from sklearn.metrics         import mean_squared_error, r2_score

In [6]:
X = load_iris().data
y = load_iris().target

df = pd.DataFrame(X, columns=load_iris().feature_names)
df['target'] = y

def bagging_classifier(df:pd.DataFrame, 
                       num_bootstrap_samples:int=3,
                       test_size:float=0.25
                      ) -> pd.DataFrame:
    
    df_train, df_test = train_test_split(df, test_size=test_size)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')

    y_pred_bagging = {}

    for i in range(num_bootstrap_samples):
        df_train = df_train.sample(n=len(df_train), 
                                   replace=True)

        X_train = df_train.drop(['target'], axis=1)
        y_train = df_train['target']
        
        model = DecisionTreeClassifier()
        model.fit(X_train, y_train)
        
        y_pred_bagging.update({i:model.predict(X_test)})

    y_pred = (pd.DataFrame(y_pred_bagging)
                .mode(axis=1)
                .rename(columns={0:'y_pred'}))

    print(model)
    print('acurácia:', accuracy_score(y_true=y_test, 
                                            y_pred=y_pred['y_pred']
                                           ))

    return pd.concat(objs=[y_test.reset_index(drop=True), 
                           y_pred['y_pred'].astype(int)], 
                     axis=1)

bagging_classifier(num_bootstrap_samples=10, df=df, test_size=0.33)

DecisionTreeClassifier()
acurácia: 0.84


,y_test,y_pred
0,0,0
1,2,2
2,2,2
3,0,0
4,1,1
5,1,1
6,0,0
7,1,1
8,0,0
9,0,0


In [7]:
X = load_diabetes().data
y = load_diabetes().target

df = pd.DataFrame(X, columns=load_diabetes().feature_names)
df['target'] = y

def bagging_regressor(df:pd.DataFrame, 
                      num_bootstrap_samples:int=3,
                      test_size:float=0.25
                      ) -> pd.DataFrame:
    
    df_train, df_test = train_test_split(df, test_size=test_size)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')
    
    y_pred_bagging = {}

    for i in range(num_bootstrap_samples):
        df_train = df_train.sample(n=len(df_train), 
                                   replace=True)

        X_train = df_train.drop(['target'], axis=1)
        y_train = df_train['target']

        model = DecisionTreeRegressor()
        model.fit(X_train, y_train)
        
        y_pred_bagging.update({i:model.predict(X_test)})

    y_pred = (pd.DataFrame(y_pred_bagging)
                .mean(axis=1)
                .rename('y_pred'))
    
    print(model)
    print('Erro quadrático médio:', mean_squared_error(y_true=y_test, 
                                                   y_pred=y_pred))
    print('Coeficiente de determinação:', r2_score(y_true=y_test, 
                                                    y_pred=y_pred))
    
    return pd.concat(objs=[y_test.reset_index(drop=True), 
                           y_pred], 
                     axis=1)
    
bagging_regressor(num_bootstrap_samples=100, df=df, test_size=0.33)

DecisionTreeRegressor()
Erro quadrático médio: 3330.834305479452
Coeficiente de determinação: 0.4581884267427726


,y_test,y_pred
0,45.0,109.68
1,262.0,225.96
2,120.0,225.02
3,84.0,120.68
4,281.0,159.02
...,...,...
141,140.0,122.56
142,111.0,109.45
143,63.0,110.64
144,116.0,97.62
